Import Libraries and choose style(s) for our future plots.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

Step 0: Import and Read Data

In [ ]:
df_reg = pd.read_csv(r'C:\Users\Kerin B\Desktop\Datasets\nba_regular_2223.csv', delimiter=',')

Step 1: Understanding Our Data
- Dataframe shape
- head and tail of data
- dtypes

In [ ]:
df_reg.shape

In [ ]:
df_reg.head()

In [ ]:
df_reg.tail(5)

In [ ]:
df_reg.columns

In [ ]:
df_reg.dtypes

Step 2: Data Preparation and Cleaning
- checking for and removing duplicates
- removing unneeded/unnecessary columns

In [ ]:
counts = df_reg['Player'].value_counts()
players_to_drop = counts[counts > 1].index
total = 'TOT'
(players_to_drop)

In [ ]:
filtered_df = df_reg[~(df_reg['Player'].isin(players_to_drop) & (df_reg['Tm'] != total))]
filtered_df.shape

In [ ]:
number_of_players = filtered_df['Player'].value_counts().sum()
unique_players = df_reg['Player'].nunique()
print('Number of Entries:', number_of_players, '-- Number of Unique Players:', unique_players)

In [ ]:
duplicates_mask = filtered_df['Player'].duplicated()
duplicates_df = filtered_df[duplicates_mask]
print("Duplicate Rows:")
(duplicates_df)

In [ ]:
filtered_df_new = filtered_df.drop(['Rk', 'Age', 'Tm', 'GS'], axis=1)

In [ ]:
filtered_df_new.set_index('Player', inplace = True)

Step 3: Creating New Columns
- MVP Eligibility based on Games Played and Average Minutes Played
- Drop Rows of Players who are not MVP Eligible

In [ ]:
criteria1 = 65
criteria2 = 20
filtered_df_new['MVP Eligibility'] = np.where((filtered_df_new['G'] >= criteria1) & (filtered_df_new['MP'] >= criteria2), 'Yes', 'No')

In [ ]:
df_reg_new = filtered_df_new[filtered_df_new['MVP Eligibility'] != 'No']

In [ ]:
filtered_df_new.head(5)

Step 4: Feature Relationships

In [ ]:
df_reg_pts = df_reg_new['PTS'].sort_values(ascending=False)
df_reg_pts.head(10).plot(kind='bar', title='Top 10 Scorers', xlabel = 'Player', ylabel = 'Points Per Game')